In [53]:
import pandas as pd
import numpy as np
import os, sys

idx = pd.IndexSlice

In [54]:
data_fp = '../data/final-data.csv'
df = pd.read_csv(data_fp, index_col=None, header=[0, 1], skipinitialspace=True)

df.rename(columns={
    'Unnamed: 0_level_1': '',
    'Unnamed: 1_level_1': '',
    'Unnamed: 2_level_1': '',
    'Unnamed: 3_level_1': ''
}, level=1, inplace=True)
df['benchmark_id'] = df.index.values

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [55]:
df.head()

model  log decomposition  SP label Cost of the alignment                    \
                                                     astar inc3 recomp-astar   
0  P430  L19   sese_manual  case_785                     0    0          0.0   
1  P430  L19   sese_manual   case_69                     0    0          0.0   
2  P430  L19   sese_manual   case_33                     0    0          0.0   
3  P430  L19   sese_manual  case_797                     4    4          4.0   
4  P430  L19   sese_manual  case_784                     0    0          0.0   

              Heuristics computed          ...       \
  recomp-inc3               astar inc3     ...        
0         0.0                   1    1     ...        
1         0.0                   1    1     ...        
2         0.0                   1    1     ...        
3         4.0                   1    1     ...        
4         0.0                   1    1     ...        

               model_trace_features                                     \
  n_uniq_tran_more_than_five_in_deg n_uniq_tran_more_than_five_out_deg   
0                                 0                                  0   
1                                 0                                  0   
2                                 0                                  0   
3                                 0                                  0   
4                                 0                                  0   

  Total Time including setup (us)                          \
                  astar_over_inc3 astar_over_recomp-astar   
0                        0.610397                0.639635   
1                        0.423914                0.473328   
2                        0.440726                0.360807   
3                        0.506494                0.659465   
4                        0.450129                0.349957   

                                                                       \
  astar_over_recomp-inc3 inc3_over_recomp-astar inc3_over_recomp-inc3   
0               0.505112               1.047901              0.827515   
1               0.350987               1.116568              0.827969   
2               0.268440               0.818665              0.609087   
3               0.499782               1.302021              0.986750   
4               0.276768               0.777459              0.614864   

                                          benchmark_id  
  recomp-astar_over_recomp-inc3  max_diff               
0                      0.789688  1.047901            0  
1                      0.741530  1.116568            1  
2                      0.744000  0.818665            2  
3                      0.757860  1.302021            3  
4                      0.790864  0.790864            4  

[5 rows x 263 columns]

In [56]:
feature_df = df.loc[:, idx[('benchmark_id', 'model_trace_features'), :]]
feature_df.rename(columns={
    '': 'benchmark_id'
}, level=1, inplace=True)
feature_df.columns = list(feature_df.columns.get_level_values(level=1))

# filter unreported features
def get_feature_cols(columns):
    not_min_max = lambda c: not c.endswith('min') and not c.endswith('max')
    not_one_deg = lambda c: not c.endswith('one_in_deg') and not c.endswith('one_out_deg')
    not_two_deg = lambda c: not c.endswith('two_in_deg') and not c.endswith('two_out_deg')
    not_three_deg = lambda c: not c.endswith('three_in_deg') and not c.endswith('three_out_deg')
    not_more_than_five_deg = lambda c: not c.endswith('five_in_deg') and not c.endswith('five_out_deg')
    joined_filter = lambda c: not_min_max(c) \
                            and not_one_deg(c) \
                            and not_two_deg(c) \
                            and not_three_deg(c) \
                            and not_more_than_five_deg(c)
    return list(filter(joined_filter, columns))

feature_cols = get_feature_cols(list(feature_df.columns))
feature_df = feature_df.loc[:, feature_cols]
reordered_cols = ['benchmark_id'] + list(filter(lambda col: col != 'benchmark_id', feature_df.columns))
feature_df = feature_df[reordered_cols]

# non_feature_colnames = df.columns.get_level_values(level=0)
# non_feature_colnames = list(filter(
#     lambda colname: colname != 'model_trace_features' \
#     and colname != 'model' \
#     and colname != 'log' \
#     and colname != 'decomposition' \
#     and colname != 'SP label' \
#     and colname != 'is_valid' \
#     and colname != 'result_dir'
# , non_feature_colnames))
performance_df = df.loc[:, idx[('benchmark_id', 'Total Time including setup (us)'), ('astar', 'inc3', 'recomp-astar', 'recomp-inc3', '')]]
performance_df.rename(columns={
#     'astar': 'astar_total_time_(us)',
#     'inc3': 'inc3_total_time_(us)',
#     'recomp-astar': 'recomp-astar_total_time_(us)',
#     'recomp-inc3': 'recomp-inc3_total_time_(us)',
    '': 'benchmark_id'
}, level=1, inplace=True)
performance_df.columns = list(performance_df.columns.get_level_values(level=1))
reordered_cols = ['benchmark_id'] + list(filter(lambda col: col != 'benchmark_id', performance_df.columns))
performance_df = performance_df[reordered_cols]

success_df = df.loc[:, idx[('benchmark_id', 'is_valid'), :]]
success_df.rename(columns={
    '': 'benchmark_id'
}, level=1, inplace=True)
success_df.columns = list(success_df.columns.get_level_values(level=1))
reordered_cols = ['benchmark_id'] + list(filter(lambda col: col != 'benchmark_id', success_df.columns))
success_df = success_df[reordered_cols]

all_df = pd.merge(left=feature_df, right=performance_df, on='benchmark_id')
all_df = pd.merge(left=all_df, right=success_df, on='benchmark_id', suffixes=('_time', '_is_valid'))
reordered_cols = ['benchmark_id'] + list(filter(lambda col: col != 'benchmark_id', all_df.columns))
all_df = all_df[reordered_cols]
all_df['min_algo'] = df.loc[:, ('Min', 'Total Time including setup (us)')]
all_df['min_time'] = performance_df[['astar', 'inc3', 'recomp-astar', 'recomp-inc3']].min(axis=1)

In [57]:
all_df.head()

,benchmark_id,trace_length,n_activity,activity_repeat_mean,activity_repeat_std,snp_n_transition,snp_n_inv_transition,snp_n_dup_transition,snp_n_uniq_transition,snp_inv_transition_in_degree_mean,...,astar_time,inc3_time,recomp-astar_time,recomp-inc3_time,astar_is_valid,inc3_is_valid,recomp-astar_is_valid,recomp-inc3_is_valid,min_algo,min_time
0,0,64.0,57.0,1.122807,0.425532,316.0,28.0,24.0,264.0,1.857143,...,22874,37474,35761,45285,True,True,True,True,astar,22874
1,1,85.0,66.0,1.287879,0.456244,358.0,28.0,76.0,254.0,1.857143,...,17436,41131,36837,49677,True,True,True,True,astar,17436
2,2,77.0,67.0,1.149254,0.500113,342.0,28.0,32.0,282.0,1.857143,...,12774,28984,35404,47586,True,True,True,True,astar,12774
3,3,148.0,85.0,1.741176,1.114332,484.0,28.0,188.0,268.0,1.857143,...,26403,52129,40037,52829,True,True,True,True,astar,26403
4,4,80.0,76.0,1.052632,0.224781,348.0,28.0,16.0,304.0,1.857143,...,13038,28965,37256,47108,True,True,True,True,astar,13038


In [58]:
two_x_diff = df.loc[:, ('Total Time including setup (us)', 'max_diff')] >= 2
filtered_df = df.loc[two_x_diff, :].reset_index()

feature_df = filtered_df.loc[:, idx[('benchmark_id', 'model_trace_features'), :]]
feature_df.rename(columns={
    '': 'benchmark_id'
}, level=1, inplace=True)
feature_df.columns = list(feature_df.columns.get_level_values(level=1))

# filter unreported features
def get_feature_cols(columns):
    not_min_max = lambda c: not c.endswith('min') and not c.endswith('max')
    not_one_deg = lambda c: not c.endswith('one_in_deg') and not c.endswith('one_out_deg')
    not_two_deg = lambda c: not c.endswith('two_in_deg') and not c.endswith('two_out_deg')
    not_three_deg = lambda c: not c.endswith('three_in_deg') and not c.endswith('three_out_deg')
    not_more_than_five_deg = lambda c: not c.endswith('five_in_deg') and not c.endswith('five_out_deg')
    joined_filter = lambda c: not_min_max(c) \
                            and not_one_deg(c) \
                            and not_two_deg(c) \
                            and not_three_deg(c) \
                            and not_more_than_five_deg(c)
    return list(filter(joined_filter, columns))

feature_cols = get_feature_cols(list(feature_df.columns))
feature_df = feature_df.loc[:, feature_cols]
reordered_cols = ['benchmark_id'] + list(filter(lambda col: col != 'benchmark_id', feature_df.columns))
feature_df = feature_df[reordered_cols]

# non_feature_colnames = df.columns.get_level_values(level=0)
# non_feature_colnames = list(filter(
#     lambda colname: colname != 'model_trace_features' \
#     and colname != 'model' \
#     and colname != 'log' \
#     and colname != 'decomposition' \
#     and colname != 'SP label' \
#     and colname != 'is_valid' \
#     and colname != 'result_dir'
# , non_feature_colnames))
performance_df = filtered_df.loc[:, idx[('benchmark_id', 'Total Time including setup (us)'), ('astar', 'inc3', 'recomp-astar', 'recomp-inc3', '')]]
performance_df.rename(columns={
#     'astar': 'astar_total_time_(us)',
#     'inc3': 'inc3_total_time_(us)',
#     'recomp-astar': 'recomp-astar_total_time_(us)',
#     'recomp-inc3': 'recomp-inc3_total_time_(us)',
    '': 'benchmark_id'
}, level=1, inplace=True)
performance_df.columns = list(performance_df.columns.get_level_values(level=1))
reordered_cols = ['benchmark_id'] + list(filter(lambda col: col != 'benchmark_id', performance_df.columns))
performance_df = performance_df[reordered_cols]

success_df = filtered_df.loc[:, idx[('benchmark_id', 'is_valid'), :]]
success_df.rename(columns={
    '': 'benchmark_id'
}, level=1, inplace=True)
success_df.columns = list(success_df.columns.get_level_values(level=1))
reordered_cols = ['benchmark_id'] + list(filter(lambda col: col != 'benchmark_id', success_df.columns))
success_df = success_df[reordered_cols]

two_x_diff_df = pd.merge(left=feature_df, right=performance_df, on='benchmark_id')
two_x_diff_df = pd.merge(left=two_x_diff_df, right=success_df, on='benchmark_id', suffixes=('_time', '_is_valid'))
reordered_cols = ['benchmark_id'] + list(filter(lambda col: col != 'benchmark_id', two_x_diff_df.columns))
two_x_diff_df = two_x_diff_df[reordered_cols]
two_x_diff_df['min_algo'] = filtered_df.loc[:, ('Min', 'Total Time including setup (us)')]
two_x_diff_df['min_time'] = performance_df[['astar', 'inc3', 'recomp-astar', 'recomp-inc3']].min(axis=1)

In [59]:
two_x_diff_df.head()

,benchmark_id,trace_length,n_activity,activity_repeat_mean,activity_repeat_std,snp_n_transition,snp_n_inv_transition,snp_n_dup_transition,snp_n_uniq_transition,snp_inv_transition_in_degree_mean,...,astar_time,inc3_time,recomp-astar_time,recomp-inc3_time,astar_is_valid,inc3_is_valid,recomp-astar_is_valid,recomp-inc3_is_valid,min_algo,min_time
0,98,104.0,72.0,1.444444,0.947776,396.0,28.0,98.0,270.0,1.857143,...,11645,74452,35257,46488,True,True,True,True,astar,11645
1,100,207.0,101.0,2.049505,1.499175,602.0,28.0,302.0,272.0,1.857143,...,43006,99682,42640,63797,True,True,True,True,recomp-astar,42640
2,404,153.0,87.0,1.758621,1.171021,494.0,28.0,194.0,272.0,1.857143,...,16961,82529,39267,55317,True,True,True,True,astar,16961
3,563,154.0,82.0,1.878049,1.510378,496.0,28.0,192.0,276.0,1.857143,...,18067,88671,38632,57720,True,True,True,True,astar,18067
4,702,160.0,89.0,1.797753,1.159739,508.0,28.0,216.0,264.0,1.857143,...,20543,82601,37908,57123,True,True,True,True,astar,20543


### Export all_df

In [60]:
outdir = '../data/python'
if not os.path.exists(outdir):
    os.mkdir(outdir)
    
all_fp = os.path.join(outdir, 'original.csv')
two_x_diff_fp = os.path.join(outdir, '2xdiff.csv')

all_df.to_csv(all_fp, index=False)
two_x_diff_df.to_csv(two_x_diff_fp, index=False)